# DeepLab Demo

This demo will demostrate the steps to run deeplab semantic segmentation model on sample input images.

## Prerequisites

Running this demo requires the following libraries:

* Jupyter notebook (Python 2)
* Tensorflow (>= v1.5.0)
* Matplotlib
* Pillow
* numpy
* ipywidgets (follow the setup [here](https://ipywidgets.readthedocs.io/en/stable/user_install.html))

In [5]:
!nvidia-smi

Sat Mar 17 09:43:56 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.12                 Driver Version: 390.12                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN Xp            Off  | 00000000:02:00.0 Off |                  N/A |
| 34%   57C    P2    71W / 250W |  11763MiB / 12196MiB |     32%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN Xp            Off  | 00000000:03:00.0 Off |                  N/A |
| 37%   62C    P2    74W / 250W |  11763MiB / 12196MiB |     30%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [10]:
%env CUDA_VISIBLE_DEVICES=0
!echo $CUDA_VISIBLE_DEVICES

env: CUDA_VISIBLE_DEVICES=0
0


## Imports

In [11]:
import collections
import os
import StringIO
import sys
import tarfile
import tempfile
import urllib

from IPython import display
from ipywidgets import interact
from ipywidgets import interactive
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

import tensorflow as tf

if tf.__version__ < '1.5.0':
    raise ImportError('Please upgrade your tensorflow installation to v1.5.0 or newer!')

# Needed to show segmentation colormap labels
sys.path.append('utils')
import get_dataset_colormap

## Select and download models

In [84]:
models = {
    'voc_mine': '/zfsauton/home/mbarnes1/deep_spectral_clustering/tensorflow/exp/VOC/train_on_train_set/train/000-semantic_Mar15-09:25:46_VCS-405b1e2_gpu8/graph.pbtxt',
    'voc_google_trainaug': '/home/scratch/mbarnes1/data/models/tensorflow/deeplabv3_pascal_train_aug/frozen_inference_graph.pb',
    'voc_google_trainval': '/home/scratch/mbarnes1/data/models/tensorflow/deeplabv3_pascal_trainval/frozen_inference_graph.pb'
}

Config = collections.namedtuple('Config', 'model_name, model_path')

def get_config(model_name):
    return Config(model_name, models[model_name])
config_widget = interactive(get_config, model_name=models.keys(), model_dir='')
display.display(config_widget)

interactive(children=(Dropdown(description=u'model_name', options=('voc_google_trainaug', 'voc_mine', 'voc_google_trainval'), value='voc_google_trainaug'), Output()), _dom_classes=('widget-interact',))

## Load model in TensorFlow

In [72]:
class DeepLabModel(object):
    """Class to load deeplab model and run inference."""
    
    INPUT_TENSOR_NAME = 'ImageTensor:0'
    OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
    INPUT_SIZE = 513

    def __init__(self, frozen_inference_graph_path):  # tarball_path):  # 
        """Creates and loads pretrained deeplab model."""
        self.graph = tf.Graph()
        
        graph_def = tf.GraphDef.FromString(open(frozen_inference_graph_path, 'rb').read())
        
        with self.graph.as_default():      
            tf.import_graph_def(graph_def, name='')
        
        self.sess = tf.Session(graph=self.graph)
            
    def run(self, image):
        """Runs inference on a single image.
        
        Args:
            image: A PIL.Image object, raw input image.
            
        Returns:
            resized_image: RGB image resized from original input image. 
            seg_map: Segmentation map of `resized_image`.
        """
        width, height = image.size
        print 'Input width {}, height {}'.format(width, height)
        resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
        target_size = (int(resize_ratio * width), int(resize_ratio * height))
        resized_image = image.convert('RGB').resize(target_size, Image.ANTIALIAS)
        print 'Resized image {}'.format(resized_image.size)
        batch_seg_map = self.sess.run(
            self.OUTPUT_TENSOR_NAME,
            feed_dict={self.INPUT_TENSOR_NAME: [np.asarray(resized_image)]})
        seg_map = batch_seg_map[0]
        return resized_image, seg_map


In [85]:
config = config_widget.result
print 'Loading model {}...'.format(config.model_name)
model = DeepLabModel(config.model_path)

Loading model voc_mine...


DecodeError: Error parsing message

## Helper methods

In [78]:
LABEL_NAMES = np.asarray([
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
    'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog',
    'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa',
    'train', 'tv'
])

FULL_LABEL_MAP = np.arange(len(LABEL_NAMES)).reshape(len(LABEL_NAMES), 1)
FULL_COLOR_MAP = get_dataset_colormap.label_to_color_image(FULL_LABEL_MAP)


def vis_segmentation(image, seg_map):
    plt.figure(figsize=(15, 5))
    grid_spec = gridspec.GridSpec(1, 4, width_ratios=[6, 6, 6, 1])

    plt.subplot(grid_spec[0])
    plt.imshow(image)
    plt.axis('off')
    plt.title('input image')
    
    plt.subplot(grid_spec[1])
    seg_image = get_dataset_colormap.label_to_color_image(
        seg_map, get_dataset_colormap.get_pascal_name()).astype(np.uint8)
    plt.imshow(seg_image)
    plt.axis('off')
    plt.title('segmentation map')

    plt.subplot(grid_spec[2])
    plt.imshow(image)
    plt.imshow(seg_image, alpha=0.7)
    plt.axis('off')
    plt.title('segmentation overlay')
    
    unique_labels = np.unique(seg_map)
    ax = plt.subplot(grid_spec[3])
    plt.imshow(FULL_COLOR_MAP[unique_labels].astype(np.uint8), interpolation='nearest')
    ax.yaxis.tick_right()
    plt.yticks(range(len(unique_labels)), LABEL_NAMES[unique_labels])
    plt.xticks([], [])
    ax.tick_params(width=0)


## Run on sample images

In [79]:
# Note that we are using single scale inference in the demo for fast
# computation, so the results may slightly differ from the visualizations
# in README, which uses multi-scale and left-right flipped inputs.

IMAGE_DIR = 'g3doc/img'

def run_demo_image(image_name):
    try:
        image_path = os.path.join(IMAGE_DIR, image_name)
        orignal_im = Image.open(image_path)
    except IOError:
        print 'Failed to read image from %s.' % image_path 
        return 
    print 'running deeplab on image %s...' % image_name
    resized_im, seg_map = model.run(orignal_im)
    
    vis_segmentation(resized_im, seg_map)

_ = interact(run_demo_image, image_name=['image1.jpg', 'image2.jpg', 'image3.jpg'])

interactive(children=(Dropdown(description=u'image_name', options=('image1.jpg', 'image2.jpg', 'image3.jpg'), value='image1.jpg'), Output()), _dom_classes=('widget-interact',))

## Run on internet images

In [50]:
def get_an_internet_image(url):
    if not url:
        return

    try:
        # Prefix with 'file://' for local file.
        if os.path.exists(url):
            url = 'file://' + url
        f = urllib.urlopen(url)
        jpeg_str = f.read()
    except IOError:
        print 'invalid url: ' + url
        return

    orignal_im = Image.open(StringIO.StringIO(jpeg_str))
    print 'running deeplab on image %s...' % url
    resized_im, seg_map = model.run(orignal_im)
    
    vis_segmentation(resized_im, seg_map)

_ = interact(get_an_internet_image, url='')

interactive(children=(Text(value=u'', description=u'url'), Output()), _dom_classes=('widget-interact',))